<a href="https://colab.research.google.com/github/riuc13/Proyecto-final/blob/main/Copia_de_AlphaFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlphaFold Colab

Este cuaderno de Colab te permite predecir fácilmente la estructura de una proteína utilizando una versión ligeramente simplificada de [AlphaFold v2.3.1](https://doi.org/10.1038/s41586-021-03819-2). 

**Diferencias con AlphaFold v2.3.1**

En comparación con AlphaFold v2.3.1, este cuaderno de Colab no utiliza plantillas (estructuras homólogas) y una parte seleccionada de la [BFD database](https://bfd.mmseqs.com/). Hemos validado estos cambios en varios miles de estructuras PDB recientes. Aunque la precisión será prácticamente idéntica al sistema completo de AlphaFold en muchos objetivos, una pequeña fracción presenta una disminución importante en la precisión debido a la MSA más pequeña y la falta de plantillas. Para mayor fiabilidad, recomendamos utilizar en [full open source AlphaFold](https://github.com/deepmind/alphafold/), o la [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/).

**Este Colab tiene una pequeña disminución en la precisión promedio para multimers en comparación con la instalación local de AlphaFold. Para una precisión completa de multímeros, se recomienda encarecidamente ejecutar  [AlphaFold locally](https://github.com/deepmind/alphafold#running-alphafold).** Además, AlphaFold-Multímero requiere buscar MSA para cada secuencia única en el complejo, por lo que es sustancialmente más lento. Si su cuaderno se agota por la búsqueda lenta de MSA de multímeros, recomendamos utilizar Colab Pro o ejecutar AlphaFold localmente.

Tenga en cuenta que este cuaderno de Colab se proporciona solo para modelado teórico y se deben tener precaución en su uso. 

El **formato de archivo PAE** se ha actualizado para coincidir con AFDB. Consulte la [AFDB FAQ](https://alphafold.ebi.ac.uk/faq/#faq-7) para obtener una descripción del nuevo formato.

**Citar esta obra**

Cualquier publicación que revele hallazgos derivados del uso de este cuaderno debería [cite](https://github.com/deepmind/alphafold/#citing-this-work) el [AlphaFold paper](https://doi.org/10.1038/s41586-021-03819-2).

**Licencias**

Este Colab utiliza los parámetros del [AlphaFold model parameters](https://github.com/deepmind/alphafold/#model-parameters-license) que están sujetos a la licencia Creative Commons Attribution 4.0 International ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode)). El Colab en sí se proporciona bajo la  [Apache 2.0 license](https://www.apache.org/licenses/LICENSE-2.0). Vea la declaración completa de licencia a continuación.

**Más información**

Puedes encontrar más información sobre cómo funciona AlphaFold en los siguientes artículos:

*   [AlphaFold methods paper](https://www.nature.com/articles/s41586-021-03819-2)
*   [AlphaFold predictions of the human proteome paper](https://www.nature.com/articles/s41586-021-03828-1)
*   [AlphaFold-Multimer paper](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1)

Las preguntas frecuentes sobre cómo interpretar las predicciones de AlphaFold son [here](https://alphafold.ebi.ac.uk/faq).

Si tiene alguna pregunta que no esté cubierta en las preguntas frecuentes, comuníquese con el equipo de AlphaFold en alphafold@deepmind.com.

**Ponerse en contacto**

Nos encantaría escuchar sus comentarios y entender cómo AlphaFold ha sido útil en su investigación. Comparte tus historias con nosotros en alphafold@deepmind.com.


## configuración

Comience ejecutando las 2 celdas a continuación para configurar AlphaFold y todo el software necesario.

In [ ]:
# Set environment variables before running any other code.
import os
os.environ['TF_FORCE_UNIFIED_MEMORY'] = '1'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '4.0'

#@title 1. Instalar software de terceros

#@markdown Ejecute esta celda presionando el botón Reproducir a la izquierda para descargar e importar software de terceros en este cuaderno de Colab.(Consulte los [acknowledgements](https://github.com/deepmind/alphafold/#acknowledgements) en nuestro archivo Léame).

#@markdown **Nota**: Esto instala el software en la computadora portátil Colab en la nube y no en su computadora.

from IPython.utils import io
import os
import subprocess
import tqdm.notebook

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      # Uninstall default Colab version of TF.
      %shell pip uninstall -y tensorflow

      %shell sudo apt install --quiet --yes hmmer
      pbar.update(6)

      # Install py3dmol.
      %shell pip install py3dmol
      pbar.update(2)

      # Install OpenMM and pdbfixer.
      %shell rm -rf /opt/conda
      %shell wget -q -P /tmp \
        https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
          && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
          && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
      pbar.update(9)

      PATH=%env PATH
      %env PATH=/opt/conda/bin:{PATH}
      %shell conda install -qy conda==4.13.0 \
          && conda install -qy -c conda-forge \
            python=3.8 \
            openmm=7.5.1 \
            pdbfixer
      pbar.update(80)

      # Create a ramdisk to store a database chunk to make Jackhmmer run fast.
      %shell sudo mkdir -m 777 --parents /tmp/ramdisk
      %shell sudo mount -t tmpfs -o size=9G ramdisk /tmp/ramdisk
      pbar.update(2)

      %shell wget -q -P /content \
        https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
      pbar.update(1)
except subprocess.CalledProcessError:
  print(captured)
  raise

In [ ]:
#@title 2. Descargar AlphaFold

#@markdown Ejecute esta celda presionando el botón *Reproducir* en
#@markdown la izquierda.

GIT_REPO = 'https://github.com/deepmind/alphafold'
SOURCE_URL = 'https://storage.googleapis.com/alphafold/alphafold_params_colab_2022-12-06.tar'
PARAMS_DIR = './alphafold/data/params'
PARAMS_PATH = os.path.join(PARAMS_DIR, os.path.basename(SOURCE_URL))

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      %shell rm -rf alphafold
      %shell git clone --branch main {GIT_REPO} alphafold
      pbar.update(8)
      # Install the required versions of all dependencies.
      %shell pip3 install -r ./alphafold/requirements.txt
      # Run setup.py to install only AlphaFold.
      %shell pip3 install --no-dependencies ./alphafold
      pbar.update(10)

      # Apply OpenMM patch.
      %shell pushd /opt/conda/lib/python3.8/site-packages/ && \
          patch -p0 < /content/alphafold/docker/openmm.patch && \
          popd

      # Make sure stereo_chemical_props.txt is in all locations where it could be searched for.
      %shell mkdir -p /content/alphafold/alphafold/common
      %shell cp -f /content/stereo_chemical_props.txt /content/alphafold/alphafold/common
      %shell mkdir -p /opt/conda/lib/python3.8/site-packages/alphafold/common/
      %shell cp -f /content/stereo_chemical_props.txt /opt/conda/lib/python3.8/site-packages/alphafold/common/

      # Load parameters
      %shell mkdir --parents "{PARAMS_DIR}"
      %shell wget -O "{PARAMS_PATH}" "{SOURCE_URL}"
      pbar.update(27)

      %shell tar --extract --verbose --file="{PARAMS_PATH}" \
        --directory="{PARAMS_DIR}" --preserve-permissions
      %shell rm "{PARAMS_PATH}"
      pbar.update(55)
except subprocess.CalledProcessError:
  print(captured)
  raise

import jax
if jax.local_devices()[0].platform == 'tpu':
  raise RuntimeError('Colab TPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')
elif jax.local_devices()[0].platform == 'cpu':
  raise RuntimeError('Colab CPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')
else:
  print(f'Running with {jax.local_devices()[0].device_kind} GPU')

# Make sure everything we need is on the path.
import sys
sys.path.append('/opt/conda/lib/python3.8/site-packages')
sys.path.append('/content/alphafold')

## Haciendo una predicción

Pegue la secuencia de su proteína en el cuadro de texto a continuación, luego ejecute las celdas restantes a través de _Runtime_ > _Run after_. También puede ejecutar las celdas individualmente presionando el botón _Reproducir_ a la izquierda.

Tenga en cuenta que la búsqueda en las bases de datos y la predicción real pueden llevar algún tiempo, de minutos a horas, según la longitud de la proteína y el tipo de GPU que Colab le asigne (consulte las preguntas frecuentes a continuación).

In [ ]:
#@title 3. Introduzca la(s) secuencia(s) de aminoácidos a plegar ⬇️
#@markdown Ingrese la(s) secuencia(s) de aminoácidos a plegar:
#@markdown * Si ingresa solo una secuencia, el modelo de monómero será
#@markdown utilizado (a menos que anule esto a continuación).
#@markdown * Si ingresa varias secuencias, se usará el modelo multimérico.

from alphafold.notebooks import notebook_utils
import enum

@enum.unique
class ModelType(enum.Enum):
  MONOMER = 0
  MULTIMER = 1

sequence_1 = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'  #@param {type:"string"}
sequence_2 = ''  #@param {type:"string"}
sequence_3 = ''  #@param {type:"string"}
sequence_4 = ''  #@param {type:"string"}
sequence_5 = ''  #@param {type:"string"}
sequence_6 = ''  #@param {type:"string"}
sequence_7 = ''  #@param {type:"string"}
sequence_8 = ''  #@param {type:"string"}
sequence_9 = ''  #@param {type:"string"}
sequence_10 = ''  #@param {type:"string"}
sequence_11 = ''  #@param {type:"string"}
sequence_12 = ''  #@param {type:"string"}
sequence_13 = ''  #@param {type:"string"}
sequence_14 = ''  #@param {type:"string"}
sequence_15 = ''  #@param {type:"string"}
sequence_16 = ''  #@param {type:"string"}
sequence_17 = ''  #@param {type:"string"}
sequence_18 = ''  #@param {type:"string"}
sequence_19 = ''  #@param {type:"string"}
sequence_20 = ''  #@param {type:"string"}

input_sequences = (
    sequence_1, sequence_2, sequence_3, sequence_4, sequence_5, 
    sequence_6, sequence_7, sequence_8, sequence_9, sequence_10,
    sequence_11, sequence_12, sequence_13, sequence_14, sequence_15, 
    sequence_16, sequence_17, sequence_18, sequence_19, sequence_20)

MIN_PER_SEQUENCE_LENGTH = 16
MAX_PER_SEQUENCE_LENGTH = 4000
MAX_MONOMER_MODEL_LENGTH = 2500
MAX_LENGTH = 4000
MAX_VALIDATED_LENGTH = 3000

#@markdown Seleccione esta casilla de verificación para ejecutar el modelo multimérico para una sola secuencia.
#@markdown Para proteínas que son monoméricas en su forma nativa, o para proteínas muy
#@markdown grandes cadenas individuales puede obtener una mayor precisión y eficiencia de la memoria
#@markdown utilizando el modelo multimérico.
#@markdown 
#@markdown 
#@markdown Debido a la mejora de la eficiencia de la memoria, el modelo multímero tiene un máximo
#@markdown límite de 4000 residuos, mientras que el modelo de monómero tiene un límite de 2500
#@markdown residuos

use_multimer_model_for_monomers = False #@param {type:"boolean"}

# Validate the input sequences.
sequences = notebook_utils.clean_and_validate_input_sequences(
    input_sequences=input_sequences,
    min_sequence_length=MIN_PER_SEQUENCE_LENGTH,
    max_sequence_length=MAX_PER_SEQUENCE_LENGTH)

if len(sequences) == 1:
  if use_multimer_model_for_monomers:
    print('Using the multimer model for single-chain, as requested.')
    model_type_to_use = ModelType.MULTIMER
  else:
    print('Using the single-chain model.')
    model_type_to_use = ModelType.MONOMER
else:
  print(f'Using the multimer model with {len(sequences)} sequences.')
  model_type_to_use = ModelType.MULTIMER

# Check whether total length exceeds limit.
total_sequence_length = sum([len(seq) for seq in sequences])
if total_sequence_length > MAX_LENGTH:
  raise ValueError('The total sequence length is too long: '
                   f'{total_sequence_length}, while the maximum is '
                   f'{MAX_LENGTH}.')

# Check whether we exceed the monomer limit.
if model_type_to_use == ModelType.MONOMER:
  if len(sequences[0]) > MAX_MONOMER_MODEL_LENGTH:
    raise ValueError(
        f'Input sequence is too long: {len(sequences[0])} amino acids, while '
        f'the maximum for the monomer model is {MAX_MONOMER_MODEL_LENGTH}. You may '
        'be able to run this sequence with the multimer model by selecting the '
        'use_multimer_model_for_monomers checkbox above.')
    
if total_sequence_length > MAX_VALIDATED_LENGTH:
  print('WARNING: The accuracy of the system has not been fully validated '
        'above 3000 residues, and you may experience long running times or '
        f'run out of memory. Total sequence length is {total_sequence_length} '
        'residues.')


In [ ]:
#@title 4. Búsqueda en bases de datos genéticas

#@markdown Una vez que se haya ejecutado esta celda, verá
#@markdown estadísticas sobre la alineación de secuencias múltiples
#@markdown (MSA) que utilizará AlphaFold. En particular,
#@markdown verá qué tan bien cada residuo está cubierto por similares
#@markdown secuencias en el MSA.

# --- Python imports ---
import collections
import copy
from concurrent import futures
import json
import random
import shutil

from urllib import request
from google.colab import files
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import py3Dmol

from alphafold.model import model
from alphafold.model import config
from alphafold.model import data

from alphafold.data import feature_processing
from alphafold.data import msa_pairing
from alphafold.data import pipeline
from alphafold.data import pipeline_multimer
from alphafold.data.tools import jackhmmer

from alphafold.common import protein

from alphafold.relax import relax
from alphafold.relax import utils

from IPython import display
from ipywidgets import GridspecLayout
from ipywidgets import Output

# Color bands for visualizing plddt
PLDDT_BANDS = [(0, 50, '#FF7D45'),
               (50, 70, '#FFDB13'),
               (70, 90, '#65CBF3'),
               (90, 100, '#0053D6')]

# --- Find the closest source ---
test_url_pattern = 'https://storage.googleapis.com/alphafold-colab{:s}/latest/uniref90_2022_01.fasta.1'
ex = futures.ThreadPoolExecutor(3)
def fetch(source):
  request.urlretrieve(test_url_pattern.format(source))
  return source
fs = [ex.submit(fetch, source) for source in ['', '-europe', '-asia']]
source = None
for f in futures.as_completed(fs):
  source = f.result()
  ex.shutdown()
  break

JACKHMMER_BINARY_PATH = '/usr/bin/jackhmmer'
DB_ROOT_PATH = f'https://storage.googleapis.com/alphafold-colab{source}/latest/'
# The z_value is the number of sequences in a database.
MSA_DATABASES = [
    {'db_name': 'uniref90',
     'db_path': f'{DB_ROOT_PATH}uniref90_2022_01.fasta',
     'num_streamed_chunks': 62,
     'z_value': 144_113_457},
    {'db_name': 'smallbfd',
     'db_path': f'{DB_ROOT_PATH}bfd-first_non_consensus_sequences.fasta',
     'num_streamed_chunks': 17,
     'z_value': 65_984_053},
    {'db_name': 'mgnify',
     'db_path': f'{DB_ROOT_PATH}mgy_clusters_2022_05.fasta',
     'num_streamed_chunks': 120,
     'z_value': 623_796_864},
]

# Search UniProt and construct the all_seq features only for heteromers, not homomers.
if model_type_to_use == ModelType.MULTIMER and len(set(sequences)) > 1:
  MSA_DATABASES.extend([
      # Swiss-Prot and TrEMBL are concatenated together as UniProt.
      {'db_name': 'uniprot',
       'db_path': f'{DB_ROOT_PATH}uniprot_2021_04.fasta',
       'num_streamed_chunks': 101,
       'z_value': 225_013_025 + 565_928},
  ])

TOTAL_JACKHMMER_CHUNKS = sum([cfg['num_streamed_chunks'] for cfg in MSA_DATABASES])

MAX_HITS = {
    'uniref90': 10_000,
    'smallbfd': 5_000,
    'mgnify': 501,
    'uniprot': 50_000,
}


def get_msa(sequences):
  """Searches for MSA for given sequences using chunked Jackhmmer search.
  
  Args:
    sequences: A list of sequences to search against all databases.

  Returns:
    A dictionary mapping unique sequences to dicionaries mapping each database
    to a list of  results, one for each chunk of the database.
  """
  sequence_to_fasta_path = {}
  # Deduplicate to not do redundant work for multiple copies of the same chain in homomers.
  for sequence_index, sequence in enumerate(sorted(set(sequences)), 1):
    fasta_path = f'target_{sequence_index:02d}.fasta'
    with open(fasta_path, 'wt') as f:
      f.write(f'>query\n{sequence}')
    sequence_to_fasta_path[sequence] = fasta_path

  # Run the search against chunks of genetic databases (since the genetic
  # databases don't fit in Colab disk).
  raw_msa_results = {sequence: {} for sequence in sequence_to_fasta_path.keys()}
  print('\nGetting MSA for all sequences')
  with tqdm.notebook.tqdm(total=TOTAL_JACKHMMER_CHUNKS, bar_format=TQDM_BAR_FORMAT) as pbar:
    def jackhmmer_chunk_callback(i):
      pbar.update(n=1)

    for db_config in MSA_DATABASES:
      db_name = db_config['db_name']
      pbar.set_description(f'Searching {db_name}')
      jackhmmer_runner = jackhmmer.Jackhmmer(
          binary_path=JACKHMMER_BINARY_PATH,
          database_path=db_config['db_path'],
          get_tblout=True,
          num_streamed_chunks=db_config['num_streamed_chunks'],
          streaming_callback=jackhmmer_chunk_callback,
          z_value=db_config['z_value'])
      # Query all unique sequences against each chunk of the database to prevent
      # redunantly fetching each chunk for each unique sequence.
      results = jackhmmer_runner.query_multiple(list(sequence_to_fasta_path.values()))
      for sequence, result_for_sequence in zip(sequence_to_fasta_path.keys(), results):
        raw_msa_results[sequence][db_name] = result_for_sequence

  return raw_msa_results


features_for_chain = {}
raw_msa_results_for_sequence = get_msa(sequences)
for sequence_index, sequence in enumerate(sequences, start=1):
  raw_msa_results = copy.deepcopy(raw_msa_results_for_sequence[sequence])

  # Extract the MSAs from the Stockholm files.
  # NB: deduplication happens later in pipeline.make_msa_features.
  single_chain_msas = []
  uniprot_msa = None
  for db_name, db_results in raw_msa_results.items():
    merged_msa = notebook_utils.merge_chunked_msa(
        results=db_results, max_hits=MAX_HITS.get(db_name))
    if merged_msa.sequences and db_name != 'uniprot':
      single_chain_msas.append(merged_msa)
      msa_size = len(set(merged_msa.sequences))
      print(f'{msa_size} unique sequences found in {db_name} for sequence {sequence_index}')
    elif merged_msa.sequences and db_name == 'uniprot':
      uniprot_msa = merged_msa

  notebook_utils.show_msa_info(single_chain_msas=single_chain_msas, sequence_index=sequence_index)

  # Turn the raw data into model features.
  feature_dict = {}
  feature_dict.update(pipeline.make_sequence_features(
      sequence=sequence, description='query', num_res=len(sequence)))
  feature_dict.update(pipeline.make_msa_features(msas=single_chain_msas))
  # We don't use templates in AlphaFold Colab notebook, add only empty placeholder features.
  feature_dict.update(notebook_utils.empty_placeholder_template_features(
      num_templates=0, num_res=len(sequence)))

  # Construct the all_seq features only for heteromers, not homomers.
  if model_type_to_use == ModelType.MULTIMER and len(set(sequences)) > 1:
    valid_feats = msa_pairing.MSA_FEATURES + (
        'msa_species_identifiers',
    )
    all_seq_features = {
        f'{k}_all_seq': v for k, v in pipeline.make_msa_features([uniprot_msa]).items()
        if k in valid_feats}
    feature_dict.update(all_seq_features)

  features_for_chain[protein.PDB_CHAIN_IDS[sequence_index - 1]] = feature_dict


# Do further feature post-processing depending on the model type.
if model_type_to_use == ModelType.MONOMER:
  np_example = features_for_chain[protein.PDB_CHAIN_IDS[0]]

elif model_type_to_use == ModelType.MULTIMER:
  all_chain_features = {}
  for chain_id, chain_features in features_for_chain.items():
    all_chain_features[chain_id] = pipeline_multimer.convert_monomer_features(
        chain_features, chain_id)

  all_chain_features = pipeline_multimer.add_assembly_features(all_chain_features)

  np_example = feature_processing.pair_and_merge(
      all_chain_features=all_chain_features)

  # Pad MSA to avoid zero-sized extra_msa.
  np_example = pipeline_multimer.pad_msa(np_example, min_num_seq=512)

In [ ]:
#@title 5. Ejecute AlphaFold y descargue la predicción

#@markdown Una vez que se ha ejecutado esta celda, un archivo zip con
#@markdown la predicción obtenida se descargará automáticamente
#@markdown a tu computador.

#@markdown En caso de que tenga problemas con la etapa de relajación, puede desactivarla a continuación.
#@markdown Advertencia: Esto significa que la predicción puede tener efectos de distracción.
#@markdown pequeñas violaciones estereoquímicas.

run_relax = False  #@param {type:"boolean"}

#@markdown La relajación es más rápida con una GPU, pero hemos descubierto que es menos estable.
#@markdown Es posible que desee habilitar la GPU para obtener un mayor rendimiento, pero si no lo hace
#@markdown converge sugerimos volver a usar sin GPU.

relax_use_gpu = False  #@param {type:"boolean"}


#@markdown El modelo de multímeros seguirá reciclando hasta que se detengan las predicciones
#@markdown cambiando, hasta el límite establecido aquí. Para una mayor precisión, en el
#@markdown costo potencial de tiempos de inferencia más largos, establezca esto en 20.

multimer_model_max_num_recycles = 3  #@param {type:"integer"}


# --- Run the model ---
if model_type_to_use == ModelType.MONOMER:
  model_names = config.MODEL_PRESETS['monomer'] + ('model_2_ptm',)
elif model_type_to_use == ModelType.MULTIMER:
  model_names = config.MODEL_PRESETS['multimer']

output_dir = 'prediction'
os.makedirs(output_dir, exist_ok=True)

plddts = {}
ranking_confidences = {}
pae_outputs = {}
unrelaxed_proteins = {}

with tqdm.notebook.tqdm(total=len(model_names) + 1, bar_format=TQDM_BAR_FORMAT) as pbar:
  for model_name in model_names:
    pbar.set_description(f'Running {model_name}')

    cfg = config.model_config(model_name)

    if model_type_to_use == ModelType.MONOMER:
      cfg.data.eval.num_ensemble = 1
    elif model_type_to_use == ModelType.MULTIMER:
      cfg.model.num_ensemble_eval = 1

    if model_type_to_use == ModelType.MULTIMER:
      cfg.model.num_recycle = multimer_model_max_num_recycles
      cfg.model.recycle_early_stop_tolerance = 0.5

    params = data.get_model_haiku_params(model_name, './alphafold/data')
    model_runner = model.RunModel(cfg, params)
    processed_feature_dict = model_runner.process_features(np_example, random_seed=0)
    prediction = model_runner.predict(processed_feature_dict, random_seed=random.randrange(sys.maxsize))

    mean_plddt = prediction['plddt'].mean()

    if model_type_to_use == ModelType.MONOMER:
      if 'predicted_aligned_error' in prediction:
        pae_outputs[model_name] = (prediction['predicted_aligned_error'],
                                   prediction['max_predicted_aligned_error'])
      else:
        # Monomer models are sorted by mean pLDDT. Do not put monomer pTM models here as they
        # should never get selected.
        ranking_confidences[model_name] = prediction['ranking_confidence']
        plddts[model_name] = prediction['plddt']
    elif model_type_to_use == ModelType.MULTIMER:
      # Multimer models are sorted by pTM+ipTM.
      ranking_confidences[model_name] = prediction['ranking_confidence']
      plddts[model_name] = prediction['plddt']
      pae_outputs[model_name] = (prediction['predicted_aligned_error'],
                                 prediction['max_predicted_aligned_error'])

    # Set the b-factors to the per-residue plddt.
    final_atom_mask = prediction['structure_module']['final_atom_mask']
    b_factors = prediction['plddt'][:, None] * final_atom_mask
    unrelaxed_protein = protein.from_prediction(
        processed_feature_dict,
        prediction,
        b_factors=b_factors,
        remove_leading_feature_dimension=(
            model_type_to_use == ModelType.MONOMER))
    unrelaxed_proteins[model_name] = unrelaxed_protein

    # Delete unused outputs to save memory.
    del model_runner
    del params
    del prediction
    pbar.update(n=1)

  # --- AMBER relax the best model ---

  # Find the best model according to the mean pLDDT.
  best_model_name = max(ranking_confidences.keys(), key=lambda x: ranking_confidences[x])

  if run_relax:
    pbar.set_description(f'AMBER relaxation')
    amber_relaxer = relax.AmberRelaxation(
        max_iterations=0,
        tolerance=2.39,
        stiffness=10.0,
        exclude_residues=[],
        max_outer_iterations=3,
        use_gpu=relax_use_gpu)
    relaxed_pdb, _, _ = amber_relaxer.process(prot=unrelaxed_proteins[best_model_name])
  else:
    print('Warning: Running without the relaxation stage.')
    relaxed_pdb = protein.to_pdb(unrelaxed_proteins[best_model_name])
  pbar.update(n=1)  # Finished AMBER relax.

# Construct multiclass b-factors to indicate confidence bands
# 0=very low, 1=low, 2=confident, 3=very high
banded_b_factors = []
for plddt in plddts[best_model_name]:
  for idx, (min_val, max_val, _) in enumerate(PLDDT_BANDS):
    if plddt >= min_val and plddt <= max_val:
      banded_b_factors.append(idx)
      break
banded_b_factors = np.array(banded_b_factors)[:, None] * final_atom_mask
to_visualize_pdb = utils.overwrite_b_factors(relaxed_pdb, banded_b_factors)


# Write out the prediction
pred_output_path = os.path.join(output_dir, 'selected_prediction.pdb')
with open(pred_output_path, 'w') as f:
  f.write(relaxed_pdb)


# --- Visualise the prediction & confidence ---
show_sidechains = True
def plot_plddt_legend():
  """Plots the legend for pLDDT."""
  thresh = ['Very low (pLDDT < 50)',
            'Low (70 > pLDDT > 50)',
            'Confident (90 > pLDDT > 70)',
            'Very high (pLDDT > 90)']

  colors = [x[2] for x in PLDDT_BANDS]

  plt.figure(figsize=(2, 2))
  for c in colors:
    plt.bar(0, 0, color=c)
  plt.legend(thresh, frameon=False, loc='center', fontsize=20)
  plt.xticks([])
  plt.yticks([])
  ax = plt.gca()
  ax.spines['right'].set_visible(False)
  ax.spines['top'].set_visible(False)
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  plt.title('Model Confidence', fontsize=20, pad=20)
  return plt

# Show the structure coloured by chain if the multimer model has been used.
if model_type_to_use == ModelType.MULTIMER:
  multichain_view = py3Dmol.view(width=800, height=600)
  multichain_view.addModelsAsFrames(to_visualize_pdb)
  multichain_style = {'cartoon': {'colorscheme': 'chain'}}
  multichain_view.setStyle({'model': -1}, multichain_style)
  multichain_view.zoomTo()
  multichain_view.show()

# Color the structure by per-residue pLDDT
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(to_visualize_pdb)
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}
if show_sidechains:
  style['stick'] = {}
view.setStyle({'model': -1}, style)
view.zoomTo()

grid = GridspecLayout(1, 2)
out = Output()
with out:
  view.show()
grid[0, 0] = out

out = Output()
with out:
  plot_plddt_legend().show()
grid[0, 1] = out

display.display(grid)

# Display pLDDT and predicted aligned error (if output by the model).
if pae_outputs:
  num_plots = 2
else:
  num_plots = 1

plt.figure(figsize=[8 * num_plots, 6])
plt.subplot(1, num_plots, 1)
plt.plot(plddts[best_model_name])
plt.title('Predicted LDDT')
plt.xlabel('Residue')
plt.ylabel('pLDDT')

if num_plots == 2:
  plt.subplot(1, 2, 2)
  pae, max_pae = list(pae_outputs.values())[0]
  plt.imshow(pae, vmin=0., vmax=max_pae, cmap='Greens_r')
  plt.colorbar(fraction=0.046, pad=0.04)

  # Display lines at chain boundaries.
  best_unrelaxed_prot = unrelaxed_proteins[best_model_name]
  total_num_res = best_unrelaxed_prot.residue_index.shape[-1]
  chain_ids = best_unrelaxed_prot.chain_index
  for chain_boundary in np.nonzero(chain_ids[:-1] - chain_ids[1:]):
    if chain_boundary.size:
      plt.plot([0, total_num_res], [chain_boundary, chain_boundary], color='red')
      plt.plot([chain_boundary, chain_boundary], [0, total_num_res], color='red')

  plt.title('Predicted Aligned Error')
  plt.xlabel('Scored residue')
  plt.ylabel('Aligned residue')

# Save the predicted aligned error (if it exists).
pae_output_path = os.path.join(output_dir, 'predicted_aligned_error.json')
if pae_outputs:
  # Save predicted aligned error in the same format as the AF EMBL DB.
  pae_data = notebook_utils.get_pae_json(pae=pae, max_pae=max_pae.item())
  with open(pae_output_path, 'w') as f:
    f.write(pae_data)

# --- Download the predictions ---
shutil.make_archive(base_name='prediction', format='zip', root_dir=output_dir)
files.download(f'{output_dir}.zip')

### Interpretando la predicción

En general, el LDDT predicho (pLDDT) se usa mejor para la confianza dentro del dominio, mientras que el error alineado predicho (PAE) se usa mejor para determinar la confianza entre dominios o entre cadenas.

Por favor vea el [AlphaFold methods paper](https://www.nature.com/articles/s41586-021-03819-2), el [AlphaFold predictions of the human proteome paper](https://www.nature.com/articles/s41586-021-03828-1), y el [AlphaFold-Multimer paper](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1) así como [our FAQ](https://alphafold.ebi.ac.uk/faq) sobre cómo interpretar las predicciones de AlphaFold.

## FAQ & Troubleshooting


*   ¿Cómo obtengo una estructura de proteína predicha para mi proteína?
    *   Haga clic en el botón _Conectar_ en la parte superior derecha para comenzar.
    *   Pegue la secuencia de aminoácidos de su proteína (sin encabezados) en "Ingrese la secuencia de aminoácidos para plegar".
    *   Ejecute todas las celdas en Colab, ya sea ejecutándolas individualmente (con el botón de reproducción en el lado izquierdo) o a través de _Tiempo de ejecución_ > _Ejecutar todo._ Asegúrese de ejecutar las 5 celdas en orden.
    *   La estructura de la proteína predicha se descargará una vez que se hayan ejecutado todas las celdas. Nota: Esto puede llevar de minutos a horas; consulte a continuación.
*   ¿Cuánto tiempo llevará esto?
    *  La descarga del código fuente de AlphaFold puede tardar unos minutos.
    *   La descarga e instalación del software de terceros puede tardar unos minutos.
    *   La búsqueda en las bases de datos genéticas puede llevar de minutos a horas.
    *   Ejecutar AlphaFold y generar la predicción puede llevar de minutos a horas, dependiendo de la longitud de su proteína y del tipo de GPU que Colab le haya asignado.
*   Mi Colab ya no parece estar haciendo nada, ¿qué debo hacer?
    *   Algunos pasos pueden tardar de minutos a horas en completarse.
    *   Si no sucede nada o si recibe un mensaje de error, intente reiniciar su tiempo de ejecución de Colab a través de _Tiempo de ejecución_ > _Reiniciar tiempo de ejecución_.
    *   Si esto no ayuda, intente restablecer su tiempo de ejecución de Colab a través de _Runtime_ > _Factory reset runtime_.
*   ¿Cómo se compara esto con la versión de código abierto de AlphaFold?
    *   Esta versión de Colab de AlphaFold busca una parte seleccionada del conjunto de datos BFD y actualmente no usa plantillas, por lo que su precisión se reduce en comparación con la versión completa de AlphaFold que se describe en el [AlphaFold paper](https://doi.org/10.1038/s41586-021-03819-2) y [Github repo](https://github.com/deepmind/alphafold/) (la versión completa está disponible a través del script de inferencia).
*   ¿Qué es Colab?
    *   Ver el [Colab FAQ](https://research.google.com/colaboratory/faq.html).
*   Recibí una advertencia "La computadora portátil requiere mucha RAM", ¿qué debo hacer?
    *  Los recursos asignados a su Colab varían. Ver el [Colab FAQ](https://research.google.com/colaboratory/faq.html) para más detalles.
    *   Sin embargo, puedes ejecutar Colab.
*   Recibí un error "El tiempo de ejecución de la CPU de Colab no es compatible" o "No se encontró GPU/TPU", ¿qué debo hacer?
    *   El tiempo de ejecución de CPU de Colab no es compatible. Intente cambiar su tiempo de ejecución a través de _runtime> Cambiar tipo de tiempo de ejecución> Acelerador de hardware> GPU.
    *  El tipo de GPU asignado a su Colab varía. Ver el [Colab FAQ](https://research.google.com/colaboratory/faq.html) para más detalles.
    *  Si recibe "No se puede conectar al backend de GPU", puede volver a intentarlo más tarde para ver si Colab le asigna una GPU.
    *   [Colab Pro](https://colab.research.google.com/signup) ofrece acceso prioritario a las GPU.
*  Recibí un error “ModuleNotFoundError: No module named...”, aunque ejecuté la celda que lo importa, ¿qué hago?
    *   Las libretas de Colab en el nivel gratuito caducan después de una cierta cantidad de tiempo. Ver el [Colab FAQ](https://research.google.com/colaboratory/faq.html#idle-timeouts). Intente volver a ejecutar todo el cuaderno desde el principio.
*   ¿Esta herramienta instala algo en mi computadora?
    *   No, todo sucede en la nube en Google Colab
    *   Al final de la ejecución de Colab, se descargará automáticamente en su computadora un archivo zip con la predicción obtenida.
*   ¿Cómo debo compartir comentarios e informes de errores?
    *   Comparta sus comentarios e informes de errores como un [issue](https://github.com/deepmind/alphafold/issues) en Github.


## Trabajo relacionado

Eche un vistazo a estos cuadernos de Colab proporcionados por la comunidad (tenga en cuenta que estos cuadernos pueden diferir de nuestro sistema AlphaFold validado y no podemos garantizar su precisión):

*   El [ColabFold AlphaFold2 notebook](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) por Sergey Ovchinnikov, Milot Mirdita y Martin Steinegger, que utiliza una API alojada en el Södinglab basada en el servidor MMseqs2 ([Mirdita et al. 2019, Bioinformatics](https://academic.oup.com/bioinformatics/article/35/16/2856/5280135)) para la creación de alineación de secuencias múltiples.


# License and Disclaimer

This is not an officially-supported Google product.

This Colab notebook and other information provided is for theoretical modelling only, caution should be exercised in its use. It is provided ‘as-is’ without any warranty of any kind, whether expressed or implied. Information is not intended to be a substitute for professional medical advice, diagnosis, or treatment, and does not constitute medical or other professional advice.

Copyright 2021 DeepMind Technologies Limited.


## AlphaFold Code License

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Model Parameters License

The AlphaFold parameters are made available under the terms of the Creative Commons Attribution 4.0 International (CC BY 4.0) license. You can find details at: https://creativecommons.org/licenses/by/4.0/legalcode


## Third-party software

Use of the third-party software, libraries or code referred to in the [Acknowledgements section](https://github.com/deepmind/alphafold/#acknowledgements) in the AlphaFold README may be governed by separate terms and conditions or license provisions. Your use of the third-party software, libraries or code is subject to any such terms and you should check that you can comply with any applicable restrictions or terms and conditions before use.


## Mirrored Databases

The following databases have been mirrored by DeepMind, and are available with reference to the following:
* UniProt: v2021\_04 (unmodified), by The UniProt Consortium, available under a [Creative Commons Attribution-NoDerivatives 4.0 International License](http://creativecommons.org/licenses/by-nd/4.0/).
* UniRef90: v2022\_01 (unmodified), by The UniProt Consortium, available under a [Creative Commons Attribution-NoDerivatives 4.0 International License](http://creativecommons.org/licenses/by-nd/4.0/).
* MGnify: v2022\_05 (unmodified), by Mitchell AL et al., available free of all copyright restrictions and made fully and freely available for both non-commercial and commercial use under [CC0 1.0 Universal (CC0 1.0) Public Domain Dedication](https://creativecommons.org/publicdomain/zero/1.0/).
* BFD: (modified), by Steinegger M. and Söding J., modified by DeepMind, available under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by/4.0/). See the Methods section of the [AlphaFold proteome paper](https://www.nature.com/articles/s41586-021-03828-1) for details.